In [1]:
import statisUtils as utils
import pandas as pd
import numpy as np
import random
from scipy.stats import loguniform, lognorm

In [2]:
df = pd.read_csv('data/MEF_QC_all.csv')
df.head()

,0,2,2.1,2.2,1,4,4.1,4.2,7,2.3,...,0.108,3.54,1.79,2.63,2.64,5.21,1.80,5.22,1.81,4.35
0,10,4,-1,-1,0,14,26,6,1,2,...,0,6,0,0,7,2,0,0,0,6
1,17,18,9,14,0,9,6,22,1,10,...,8,24,22,12,14,27,16,22,2,12
2,1,0,4,9,0,0,-1,0,-1,0,...,6,4,1,-1,2,5,11,3,0,3
3,0,0,4,0,2,10,0,2,8,0,...,2,3,5,3,0,3,0,5,3,0
4,1,8,2,5,10,11,17,3,1,10,...,6,3,3,1,4,5,1,0,4,6


In [3]:
data = np.array(df)
data

array([[10,  4, -1, ...,  0,  0,  6],
       [17, 18,  9, ..., 22,  2, 12],
       [ 1,  0,  4, ...,  3,  0,  3],
       ...,
       [ 1,  2,  4, ...,  0,  4,  9],
       [ 6, 24, 21, ..., 13,  7,  1],
       [ 0, 12,  2, ...,  2,  2,  0]], dtype=int64)

# using code from test_MEFs.m file#

In [4]:
# global statistics 
data_mean = np.mean(data)
data_var = np.var(data)
data_noise = data_var/(data_mean**2)

# vals 7.4, 259.0, 4.7

In [5]:
# get 6 observerd moments for all genes
statis_data = utils.statisData(data)

In [6]:
# define Discrepancy metrics (rho) as placeholder(lambda) function
rho = lambda s: sqrt(np.sum(log(statis_data/s)**2))

In [7]:
# define f as model statistics function
f = lambda k: statisGTM(k,4)

In [8]:
N = 1000 # the number of particles
T = 5 # number of rounds
epsilon = 1# a sequence of discrepancy acceptance thresholds

In [9]:
# define prior as anonymous lambda function
# [kon ron koff roff mu delta]
# OFF dwell time distribution f_off(t) = ron^(kon) * t^(kon-1) * e^(-ron * t) / gamma(kon)
# ON dwell time distribution f_on(t) = roff^(koff) * t^(koff-1) * e^(-roff * t) / gamma(koff)

prior = np.array([
                random.uniform(0,5), # kon ~ U[0,5]
                loguniform(10**-1, 10**1).rvs(), # ron ~ logU[-1,1] base 10
                random.uniform(0,5), # koff ~ U[0,5]
                loguniform(10**-1, 10**1).rvs(), # roff ~ logU[-1,1] base 10 
                random.uniform(0,50), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
                1]) # rdeg=1

In [10]:
proposal_sigma = 0.2

In [11]:
# define proposal as lambda function
# proposal kernel process, generates new trials
proposal = lambda x: np.random.lognormal(x,proposal_sigma)

In [12]:
def lognpdf(x, mu, sigma):
    shape  = sigma
    loc    = 0
    scale  = np.exp(mu)
    return lognorm.pdf(x, shape, loc, scale)

In [13]:
# define lamda function for proposal_pdf
# the PDF of the proposal kernel
proposal_pdf = lambda kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior: \
    lognpdf(mu_post,np.log(mu_prior),proposal_sigma) * lognpdf(kon_post,np.log(kon_prior),proposal_sigma) \
    * lognpdf(ron_post,np.log(ron_prior),proposal_sigma) * lognpdf(koff_post,np.log(koff_prior),proposal_sigma) \
    * lognpdf(roff_post,np.log(roff_prior),proposal_sigma) 

In [14]:
test = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
test.shape[1]

3

In [15]:
np.ones((4,3)) / 4

array([[0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25]])